In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [2]:
#open chrome driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/charlie/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


<h1 style="color:blue">Scraping first headline </h1>

In [3]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html


soup = bs(html, 'html.parser')

start = soup.find('li', class_="slide")
title = start.find('div', class_='content_title').text.strip()

title

'Another First: Perseverance Captures the Sounds of Driving on Mars'

<h1 style="color:blue">Scraping first teaser paragraph </h1>

In [4]:
news_start = soup.find("li", class_= "slide")
news_p = news_start.find('div', class_="article_teaser_body").text
news_p

'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'

<h1 style="color:blue"> JPL Mars Space Images - Featured Image </h1>

In [5]:
jpl_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(jpl_url)

html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
images = soup.find_all('div', class_='floating_text_area')

for image in images:
    link = image.find("a")
    href = link['href']
    
    featured_image_url = ("https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/" + href)
    
featured_image_url
    

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'

<h1 style="color:blue">Table Facts</h1>

In [17]:
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)

mars_table_df = tables[0]
mars_tableII = mars_table_df.set_index(0, inplace=True)

mars_tableIII = mars_table_df.rename(columns={0:' ',1:' '})

mars_html_table = mars_tableIII.to_html()

mars_html_table_clean = mars_html_table.replace("\n", "")

mars_html_table_clean

mars_data['mars_html_table_clean'] = mars_html_table_clean



NameError: name 'mars_data' is not defined

In [ ]:
# mars_table_II = mars_table_df.rename(columns={"0": "Mars Planet Profile", "1" : "Facts"}, inplace=True)
# mars_table_II.head()

# mars_html_table = mars_table_df.to_html()
# mars_html_table

# mars_html_table = mars_html_table.replace('\n', '')

# mars_html_table

# facts_url = 'https://space-facts.com/mars/'
# tables = pd.read_html(facts_url)
# tables
# mars_table_df = tables[0]
# mars_table_df.head()





<h1 style="color:blue"> Mars Hemispheres </h1>

In [9]:
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

<strong><h1 style="color:red">DO NOT DELETE BELOW</h1></strong>

In [41]:
browser.visit(hemi_url)
html = browser.html
soup = bs(html, 'html.parser')

hemi_main_url = "https://astrogeology.usgs.gov"
full_url_dict = []

hemi_names = soup.find_all('div', class_='item')

for i in hemi_names:
    title_hemi = i.find("h3").text.strip()
    link = i.find('a')['href']
    enhanced_img_link = hemi_main_url + link
    #move to page with full picture
    browser.visit(enhanced_img_link)
    html = browser.html
    soup = bs(html, 'html.parser')
    enhanced_img = soup.find("div", class_="downloads")
    final_link = enhanced_img.find('a')['href']
    
    
    full_image_url_link = (hemi_main_url + final_link)
#     print(title)
#     print(full_image_url_link)
    
    full_url_dict.append({
        "Hemisphere Image" : title_hemi, 
        "URL" : final_link
    })
    
full_url_dict


[{'Hemisphere Image': 'Cerberus Hemisphere Enhanced',
  'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Hemisphere Image': 'Schiaparelli Hemisphere Enhanced',
  'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Hemisphere Image': 'Syrtis Major Hemisphere Enhanced',
  'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Hemisphere Image': 'Valles Marineris Hemisphere Enhanced',
  'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [18]:
mars_data =({

    'title': title,
    'news_p': news_p,
    'featured_image_url': featured_image_url,
    'mars_html_table': mars_html_table,
    'full_url_dict': full_url_dict
})

mars_data


NameError: name 'full_url_dict' is not defined

In [49]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [54]:
db = client.mmars_db

In [55]:
marrs = db.mmars_data

In [56]:
results = marrs.find()
for result in results:
    print(result)
    print("-------")

{'_id': ObjectId('604ed62eacfa175e7b04ac8a'), 'title': "NASA's Perseverance Mars Rover Mission Honors Navajo Language", 'news_p': 'Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.', 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg', 'full_url_dict': [{'Hemisphere Image': 'Cerberus Hemisphere Enhanced', 'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Hemisphere Image': 'Schiaparelli Hemisphere Enhanced', 'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'Hemisphere Image': 'Syrtis Major Hemisphere Enhanced', 'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'Hemisphere Image': 'Valles Marineris Hemisphere Enhanced', 'URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_mariner